## Setup

In [44]:
# disabling contracts for speed
import contracts
import yaml
contracts.disable_all()

In [45]:
import duckietown_world as dw

In [46]:
# reducing the verbosity to critical
dw.logger.setLevel(50)

Utility function to draw in IPython:

In [47]:
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_svg, ipython_draw_html

## Pose interpolation code

In [48]:
import geometry as geo
import numpy as np

In [49]:
def interpolate(q0, q1, alpha):
    v = geo.SE2.algebra_from_group(geo.SE2.multiply(geo.SE2.inverse(q0), q1))
    vi = v * alpha
    q = np.dot(q0, geo.SE2.group_from_algebra(vi))
    return q

## PlacedObject example

All objects in the map are instances of `PlacedObject`. 

To create a new object, subclass `PlacedObject` and implement the drawing method `draw_svg` 
and the `extent_points` method.

The SVG drawing is done using the [`svgwrite`][svgwrite] library.

[svgwrite]: https://svgwrite.readthedocs.io/en/master/

In [50]:
class Person(dw.PlacedObject):

    def __init__(self, radius, *args, **kwargs):
        self.radius = radius
        dw.PlacedObject.__init__(self, *args, **kwargs)
        
    def draw_svg(self, drawing, g):
        # drawing is done using the library svgwrite
        c = drawing.circle(center=(0, 0), r=self.radius, fill='pink')
        g.add(c)
        # draws x,y axes
        dw.draw_axes(drawing, g)

    def extent_points(self):
        # set of points describing the boundary 
        L = self.radius
        return [(-L, -L), (+L, +L)]


## Animation example

Create the interpolated poses:

In [51]:
q0 = geo.SE2_from_translation_angle([0, 0], 0)
q1 = geo.SE2_from_translation_angle([2, -2], np.deg2rad(-90))
print(q1)
# create a sequence of poses 
n = 10
seqs = []
steps = np.linspace(0, 1, num=n)
for alpha in steps:
    q = interpolate(q0, q1, alpha)
    seqs.append(q)


[[ 6.123234e-17  1.000000e+00  2.000000e+00]
 [-1.000000e+00  6.123234e-17 -2.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00]]


Create a root PlacedObject:

In [52]:
root = dw.PlacedObject()
import sys
import traceback





with open("autobot29.yaml", 'r') as stream:

    try:

        data = yaml.safe_load(stream)

    except yaml.YAMLError as exc:

        print(exc)

    

    

    data_points = len(data['trajectory_data'])

    x = np.zeros((data_points,))

    y= np.zeros((data_points,))

    R = np.zeros((3,3, data_points))            
    phi = np.zeros((3, data_points))

    

    dx = 999.999*np.ones((data_points, ))

    dy = 999.999*np.ones((data_points, ))

    dr = 999.999*np.ones((data_points, ))
    dphi = 999.999*np.ones((data_points, ))

    final_trajectory = []

    for idx, [time, traj] in enumerate(data['trajectory_data'].items()):

        x[idx] = np.array(traj[0])

        y[idx] = np.array(traj[1])

        R[:,:,idx] = np.reshape(np.array(traj[3:]), (3,3))

    

        phi[:,idx] = np.array([

                np.arctan2(-R[1,2,idx],R[2,2,idx]), 

                np.arctan2(R[0,2,idx],np.sqrt(R[0,0,idx]**2 + R[0,1,idx]**2)),

                np.arctan2(-R[0,1,idx], R[0,0,idx])])

    

        #print(phi[:,idx])
        z = phi[2,idx]
        points = np.array([x[idx], y[idx]])
        final_trajectory.append([points, z])
    final_array = final_trajectory
seqs2 = []
for entry in range(0, len(final_array)):
    x = final_array[entry][0][0]
    y = final_array[entry][0][1]
    alpha = final_array[entry][1]
    q5 = geo.SE2_from_translation_angle([x,y],alpha)
    seqs2.append(q5)


Create a SampledSequence of the pose:

In [53]:
timestamps = range(len(seqs2)) # [0, 1, 2, ...]
print(seqs2)
# SE2Transform is the wrapper for SE2 used by Duckietown World 
transforms = [dw.SE2Transform.from_SE2(_) for _ in seqs2]
print(transforms)
seq_me = dw.SampledSequence(timestamps, transforms)

    
print(seq_me.timestamps)

print(seq_me.values[0])

[array([[-0.61727686,  0.786746  , -0.33      ],
       [-0.786746  , -0.61727686,  2.194     ],
       [-0.        ,  0.        ,  1.        ]]), array([[-0.61734933,  0.78668914, -0.333     ],
       [-0.78668914, -0.61734933,  2.189     ],
       [-0.        ,  0.        ,  1.        ]]), array([[-0.62078234,  0.78398296, -0.334     ],
       [-0.78398296, -0.62078234,  2.187     ],
       [-0.        ,  0.        ,  1.        ]]), array([[-0.62287261,  0.78232328, -0.335     ],
       [-0.78232328, -0.62287261,  2.187     ],
       [-0.        ,  0.        ,  1.        ]]), array([[-0.62243267,  0.78267335, -0.335     ],
       [-0.78267335, -0.62243267,  2.188     ],
       [-0.        ,  0.        ,  1.        ]]), array([[-0.62121816,  0.78363767, -0.336     ],
       [-0.78363767, -0.62121816,  2.186     ],
       [-0.        ,  0.        ,  1.        ]]), array([[-0.61649753,  0.78735684, -0.341     ],
       [-0.78735684, -0.61649753,  2.181     ],
       [-0.        ,  0.   

Add the object `me` to the root, saying it's a Person, and with the 
sequence above given as ground truth for the pose.

In [54]:

root.set_object("me", Person(0.5), ground_truth=seq_me)

Finally, draw the animation:

In [58]:
area = dw.RectangularArea((-4, -4), (4, 4))
m = dw.load_map('udem1')
ipython_draw_html(m);


Let's now get the lane object:

In [59]:
from duckietown_world.world_duckietown.tile_template import load_tile_types

In [89]:
print(m['tilemap'])
wantedTile = m['tilemap']
print(wantedTile[3])
for entry in wantedTile:
    template = load_tile_types()['curve_left']
    from copy import deepcopy
    lane_segment = deepcopy(template['curve/lane2'])
    break

TileMap(children={'tile-0-6': Tile(children={}, spatial_relations={}), 'tile-1-6': Tile(children={}, spatial_relations={}), 'tile-2-6': Tile(children={}, spatial_relations={}), 'tile-3-6': Tile(children={}, spatial_relations={}), 'tile-4-6': Tile(children={}, spatial_relations={}), 'tile-5-6': Tile(children={}, spatial_relations={}), 'tile-6-6': Tile(children={}, spatial_relations={}), 'tile-7-6': Tile(children={}, spatial_relations={}), 'tile-0-5': Tile(children={}, spatial_relations={}), 'tile-1-5': Tile(children={'curve_left': PlacedObject(children={'curve': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],1.5707963267948966))})}, spatial_relations={'curve': GroundTruth(() -> ('curve',)  SE2Transform([0.0, 0.0],4.71238898038469))})}, spatial_rela

KeyError: 'Tile "3" not available'

We can use the function `lane_segment.lane_pose_from_SE2Transform` to get the 
lane pose information (relative heading, etc.), including the projection
to the midlane.

In [65]:
center_points = []

for timestamp, pose_object in seq_me:
    lane_pose = lane_segment.lane_pose_from_SE2Transform(pose_object)
    print(lane_pose.distance_from_center, timestamp)
    
    center_points.append(lane_pose.center_point)

sequence = dw.SampledSequence(seq_me.timestamps, center_points)


2.4139999986526015 0.0
2.4089999986526016 1.0
2.4069999986526014 2.0
2.4069999986526014 3.0
2.4079999986526017 4.0
2.4059999986526015 5.0
2.4009999986526016 6.0
2.3909999986526014 7.0
2.3789999986526014 8.0
2.3659999986526015 9.0
2.3509999986526013 10.0
2.3339999986526014 11.0
2.3189999986526018 12.0
2.3039999986526016 13.0
2.2849999986526015 14.0
2.2619999986526014 15.0
2.2419999986526014 16.0
2.2249999986526015 17.0
2.2219999986526013 18.0
2.2229999986526017 19.0
2.2219999986526013 20.0
2.2189999986526017 21.0
2.2119999986526016 22.0
2.2019999986526018 23.0
2.1909999986526016 24.0
2.1819999986526017 25.0
2.1829999986526016 26.0
2.1819999986526017 27.0
2.180999998652602 28.0
2.172999998652602 29.0
2.1619999986526017 30.0
2.146999998807907 31.0
2.128999998807907 32.0
2.1089999988079073 33.0
2.090999998807907 34.0
2.0769999988079073 35.0
2.056999998807907 36.0
2.037999998807907 37.0
2.013999998807907 38.0
1.9959999988079071 39.0
1.974999998807907 40.0
1.953999998807907 41.0
1.9269999988

1.4391289712648507 350.0
1.4377185790726208 351.0
1.4323016179193135 352.0
1.4235152370082653 353.0
1.410629665831046 354.0
1.3903270148169535 355.0
1.3721274504813687 356.0
1.3546348828958974 357.0
1.3345178170243248 358.0
1.3150854862520232 359.0
1.2991666235145447 360.0
1.2761714637926906 361.0
1.2696756378341654 362.0
1.26607949578926 363.0
1.2645006954566065 364.0
1.262922651361756 365.0
1.262922651361756 366.0
1.262922651361756 367.0
1.262922651361756 368.0
1.262922651361756 369.0
1.262922651361756 370.0
1.262738646999141 371.0
1.261174123384622 372.0
1.261174123384622 373.0
1.261174123384622 374.0
1.261174123384622 375.0
1.2617908476030462 376.0
1.261174123384622 377.0
1.261174123384622 378.0
1.262738646999141 379.0
1.2643261421269454 380.0
1.2659040790291698 381.0
1.2674827712591201 382.0
1.269239490653433 383.0
1.270029975293384 384.0
1.26985223006181 385.0
1.26985223006181 386.0
1.26985223006181 387.0
1.26985223006181 388.0
1.26985223006181 389.0
1.26985223006181 390.0
1.2698

In [66]:
# we now add a marker for projection in the center point

#ane_segment.set_object("projection2", dw.PlacedObject(), ground_truth=sequence)
lane_segment.set_object("me", Person(0.2), ground_truth=sequence)


In [67]:
ipython_draw_html(lane_segment);